In [ ]:
'''
掛載Google硬碟
安裝套件
引用套件
APP應用準備
消息準備
handler執行方法設計
啟動應用

'''

In [ ]:
'''

資料 mapping 至google drive

把資料寫在/content/drive

即可保存在 google drive內

'''
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
'''

流程解析

'''


In [ ]:
!pip install line-bot-sdk flask flask-ngrok

In [ ]:
'''
引用套件
'''

# 引用Web Server套件
from flask import Flask, request, abort, jsonify

# 載入json處理套件
import json

# 外部連結自動生成套件
from flask_ngrok import run_with_ngrok

# 從linebot 套件包裡引用 LineBotApi 與 WebhookHandler 類別
from linebot import (
    LineBotApi, WebhookHandler
)

# 引用無效簽章錯誤
from linebot.exceptions import (
    InvalidSignatureError
)


In [ ]:
'''
建置主程序

建置handler與 line_bot_api
'''

# 設定Server啟用細節
app = Flask(__name__,static_url_path = "/material" , static_folder = "./material/")
run_with_ngrok(app)

# 生成實體物件，跟line溝通用
line_bot_api = LineBotApi("your channel access token")
# 收發消息專用
handler = WebhookHandler("your channel secret")

In [ ]:
'''
建置主程序的API入口
  接受Line傳過來的消息
  並取出消息內容
  將消息內容存在google drive的檔案內
  並請handler 進行消息驗證與轉發
'''


# 啟動server對外接口，使Line能丟消息進來
@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    print(body)

    # 記錄用戶log
    f = open("/content/drive/MyDrive/ai-event.log", "a")
    f.write(body)
    f.close()

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

In [ ]:
'''
QuickReply 快速回覆，有很多小按鍵，按鍵使用後就會消失
準備 QuickReply 的 Button，必須先把按鍵準備好
'''

# 引入相關套件
from linebot.models import (
    MessageAction, URIAction,
    PostbackAction, DatetimePickerAction,
    CameraAction, CameraRollAction, LocationAction,
    QuickReply, QuickReplyButton
)

# 創建QuickReplyButton 

# 點擊後，以用戶身份發送文字消息
# QuickReplyButton，action 填 Action，MessageAction是以用戶身分發文字，label 是按鍵的文字，text是以用戶角度講的話
textQuickReplyButton = QuickReplyButton(
    action=MessageAction(
        label="發送文字消息", 
        text="text2"
    )
)

## 點擊後，彈跳出選擇時間之視窗
## DatetimePickerAction
# label是按鍵的恩茲，data是隱藏資料，mode data是用戶輸入的畫面是天數
dateQuickReplyButton = QuickReplyButton(
    action=DatetimePickerAction(
        label="時間選擇",
        data="data3",                       
        mode="date"
    )
)


## 點擊後，開啟相機
## CameraAction
cameraQuickReplyButton = QuickReplyButton(
    action = CameraAction(label="拍照")
)

## 點擊後，切換至照片相簿選擇。可以讓用戶發圖片消息給我們
cameraRollQRB = QuickReplyButton(
    action = CameraRollAction(label="選擇照片")
)

## 點擊後，跳出地理位置。可以引導用戶發地理位置消息給我們
locationQRB = QuickReplyButton(
    action = LocationAction(label="地理位置")
)

## 點擊後，以Postback事件回應Server。利用data做很多特殊功能
postbackQRB = QuickReplyButton(
    action=PostbackAction(label="我是PostbackEvent", data="Data1")
)

In [ ]:
'''

以QuickReply封裝該些QuickReply Button

'''
## 設計 QuickReplyButton 的 List，list裡面放那些按鍵
quickReplyList = QuickReply(
    items = [textQuickReplyButton, dateQuickReplyButton, cameraQuickReplyButton, cameraRollQRB, locationQRB,postbackQRB]
)

In [ ]:
'''

製作TextSendMessage，並將剛封裝的QuickReply放入

'''
## 將quickReplyList 塞入TextSendMessage 中 
from linebot.models import (
    TextSendMessage,
)
# quickreply通常必須夾帶在xxxSendMessage的裡面，都會有quick_reply參數，此參數就是放QuickReply
quick_reply_text_send_message = TextSendMessage(text='發送問題給用戶，請用戶回答', quick_reply=quickReplyList)

In [ ]:
'''

設計一個字典
    當用戶輸入相應文字消息時，系統會從此挑揀消息

'''

# 將消息模型，文字收取消息與文字寄發消息 引入
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage, ImageSendMessage
)

# 根據自定義菜單四張故事線的圖，設定相對應image
template_message_dict = {
  "@reply":quick_reply_text_send_message,
}

In [ ]:
#用戶點擊button後，觸發postback event，對其回傳做相對應處理
from linebot.models import PostbackEvent

@handler.add(PostbackEvent)
def handle_post_message(event):
    user_profile = line_bot_api.get_profile(event.source.user_id)
    if (event.postback.data.find('Data1')== 0):
        with open("user_profile_business.txt", "a") as myfile:
            myfile.write(json.dumps(vars(user_profile),sort_keys=True))
            myfile.write('\n')
            line_bot_api.reply_message(
            event.reply_token,
                TextSendMessage(
                    text='請稍待，會有專人與您聯繫'
                )
            )
    elif (event.postback.data.find('Data2') == 0):
        with open("user_profile_tutorial.txt", "a") as myfile:
            myfile.write(json.dumps(vars(user_profile),sort_keys=True))
            myfile.write('\n')
            line_bot_api.reply_message(
            event.reply_token,
                TextSendMessage(
                    text='請稍待，我們會派專家您聯繫'
                )
            )
    else:
        pass

In [ ]:
# 用戶發出文字消息時， 按條件內容, 回傳文字消息
# 為了要觸發字典裡的消息，設計當用戶發文字，就會回字典裡面找消息
# 記得傳$reply給機器人
@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    
    if(event.message.text.find('@')!= -1):
        line_bot_api.reply_message(
        event.reply_token,
        template_message_dict.get(event.message.text)
        )
    else:
        line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text="字典內查無此字，請輸入@more")
        )

In [ ]:
# 主程序運行
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://4a56-35-231-143-110.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
{"destination":"Ude2ebddaef1e1b50ddbe960f037b8efe","events":[{"type":"message","message":{"type":"text","id":"16910459280152","text":"@reply"},"webhookEventId":"01GERKRK73ZT3887SHFASKFWNF","deliveryContext":{"isRedelivery":false},"timestamp":1665125862120,"source":{"type":"user","userId":"U64e856315f80175e21257c6e7eb77412"},"replyToken":"91572cce48a54b819107d369dd8387f4","mode":"active"}]}


INFO:werkzeug:127.0.0.1 - - [07/Oct/2022 06:57:43] "POST / HTTP/1.1" 200 -


{"destination":"Ude2ebddaef1e1b50ddbe960f037b8efe","events":[{"type":"message","message":{"type":"text","id":"16910467366008","text":"text2"},"webhookEventId":"01GERKVX51QJXSEFVBF129C4XB","deliveryContext":{"isRedelivery":false},"timestamp":1665125970702,"source":{"type":"user","userId":"U64e856315f80175e21257c6e7eb77412"},"replyToken":"ad29eeef81464c3194501c56f16933d0","mode":"active"}]}


...

INFO:werkzeug:127.0.0.1 - - [07/Oct/2022 06:59:47] "POST / HTTP/1.1" 200 -


{"destination":"Ude2ebddaef1e1b50ddbe960f037b8efe","events":[{"type":"postback","postback":{"data":"Data1"},"webhookEventId":"01GERKWZEJ9KVF3XM6KRSTV44W","deliveryContext":{"isRedelivery":false},"timestamp":1665126006158,"source":{"type":"user","userId":"U64e856315f80175e21257c6e7eb77412"},"replyToken":"6721a848b17044beaacc832c5f7e16d2","mode":"active"}]}


INFO:werkzeug:127.0.0.1 - - [07/Oct/2022 07:00:07] "POST / HTTP/1.1" 200 -
ERROR:__main__:Exception on / [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/usr/local/lib/python3.7/dist-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "<ipython-input-5-76ebdfe53b31>", line 27, in callback
 

{"destination":"Ude2ebddaef1e1b50ddbe960f037b8efe","events":[{"type":"message","message":{"type":"text","id":"16910492062249","text":"@more"},"webhookEventId":"01GERM5NSJ0YWMM8MZ2TVNR7CX","deliveryContext":{"isRedelivery":false},"timestamp":1665126290761,"source":{"type":"user","userId":"U64e856315f80175e21257c6e7eb77412"},"replyToken":"1af59cee9dc94603be563f0cbe32dbef","mode":"active"}]}
{"destination":"Ude2ebddaef1e1b50ddbe960f037b8efe","events":[{"type":"message","message":{"type":"text","id":"16910502040173","text":"@reply"},"webhookEventId":"01GERM9K5XPNSN7NMK3GQKHJ3F","deliveryContext":{"isRedelivery":false},"timestamp":1665126419527,"source":{"type":"user","userId":"U64e856315f80175e21257c6e7eb77412"},"replyToken":"d80b43ab954d4575a5861745db2990ed","mode":"active"}]}


INFO:werkzeug:127.0.0.1 - - [07/Oct/2022 07:07:00] "POST / HTTP/1.1" 200 -
